## Process Description

![Dietician](Assets/Simpy_Triage.png)

Simulate a process of people visiting a hospital and sent to ambulatory care (i.e. out-patient department) or to emergency care after triage. People visit the clinic, register at the front desk, then go to triage after which 20 percent are processed in ambulatory care and 80 percent assessed in emergency care.

1. **Entity**: People arriving for treatment
2. **Generator**: Generate arrivals  
3. **Inter-Arrrival Time**: An exponential distribution sampled for patient arrivals  
4. **Activity**: Registration followed by triage followed by assessment in ambulatory care (20 percent) or emergency care (80 percent)
5. **Activity Time**: Exponential distributions
6. **Resources**: Receptionist, nurse, doctor OPD, doctor ER
7. **Queues**: People waiting for registration after arrival, for triage after registration and for assessment after assignment to OPD or ER.
8. **Sink**: Exit after assessment.

Based on: https://youtu.be/QV-pJnKrGuc

### 1. Import packages

Aside from **simpy**, we need **random** to sample from random distributions.

In [28]:
import simpy
import random
from numpy import median
from matplotlib import pyplot as plt


### 2. Generate Arrivals

Define the generator for arrivals. This puts patients in the queue for consultation.

**Args**:
- env (Simpy environment) required
- mean_IAT (int) mean Inter-Arrival Time for generating arrivals
- meanCT2register (int) mean Cycle Time for registration
- meanCT2triage (int) mean Cycle Time for triage
- meanCT2assessnOPD (int) mean Cycle Time for assessment in OPD
- meanCT2assessnER (int) mean Cycle Time for assessment in ER
- receptionist (Simpy resource) resource
- nurse (Simpy resource) resource
- doctorOPD (Simpy resource) resource
- doctorER (Simpy resource) resource

**Notes**:
- Has a forever-while loop in which it:
	- Creates a patient ID and sends the patient onward on their journey, invoking the function that generates consultations with the patient's ID.
	- Has one ```yield``` statement to timeout for sampled Inter-Arrival Time, after which it increments patient ID. The ```yield``` with timeout will await the action (i.e. completion of timer), freezing until then and next resuming where it left off.

In [29]:
def generate_arrivals(env, mean_IAT, meanCT2register, meanCT2triage, meanCT2assessnOPD, meanCT2assessnER, 
                        receptionist, nurse, doctorOPD, doctorER):
    # Patient zero
    patient_ID = 0

    while True:
        # Send patient onward 
        generator_process_model = generate_process_model(env, meanCT2register, meanCT2triage, meanCT2assessnOPD, meanCT2assessnER, 
                        receptionist, nurse, doctorOPD, doctorER, patient_ID)
        env.process(generator_process_model)

        # Wait for next arrival
        delta4arrival = random.expovariate(1.0 / mean_IAT)
        yield env.timeout(delta4arrival)

        # Next patient
        patient_ID += 1

### 3. Generate Action

Define the generator for the process model.

**Args**:
- env (Simpy environment) required
- meanCT2register (int) mean Cycle Time for registration
- meanCT2triage (int) mean Cycle Time for triage
- meanCT2assessnOPD (int) mean Cycle Time for assessment in OPD
- meanCT2assessnER (int) mean Cycle Time for assessment in ER
- receptionist (Simpy resource) resource
- nurse (Simpy resource) resource
- doctorOPD (Simpy resource) resource
- doctorER (Simpy resource) resource
- patient_ID (int) patient ID from the function that generates arrivals

**Notes**: 
- For each activity in the process model, has two ```with``` declarations as follows:
    - Request the resource and ```yield``` until available
    - Execute the activity by timing out for sampled activity time. 
- Records timestamps at: arrived, queued for activity, started activity, exit
- The fork in the road is implemented by conditional logic that samples a uniform random distribution

In [30]:
def generate_process_model(env, meanCT2register, meanCT2triage, meanCT2assessnOPD, meanCT2assessnER, 
                        receptionist, nurse, doctorOPD, doctorER, patient_ID):
    global queued4registration
    global queued4triage
    global queued4assessmentOPD
    global queued4assessmentER
    global leadTimes

    arrived = env.now
    print("{} arrived at {:.2f}".format(patient_ID, arrived))

    # Request a receptionist for registration
    with receptionist.request() as req:
        # Wait until receptionist is available
        yield req

        started = env.now
        queued = started - arrived
        queued4registration.append(queued)
        print("{} started registration at {:.2f} after waiting {:.2f}".format(patient_ID, started, queued))
        
        delta = random.expovariate(1.0 / mean_CT2register)
        yield env.timeout(delta)

        arrived4triage = env.now

    with nurse.request() as req:
        # Wait until nurse is available
        yield req
        
        started = env.now
        queued = started - arrived4triage
        queued4triage.append(queued)
        print("{} started triage at {:.2f} after waiting {:.2f}".format(patient_ID, started, queued))

        delta = random.expovariate(1.0 / mean_CT2triage)
        yield env.timeout(delta)

        arrived4assessment = env.now

    which_way = random.uniform(0, 1)

    if (which_way < 0.2):
        with doctorOPD.request() as req:
            # Wait until doctor is available in outpatient care
            yield req

            started = env.now
            quued = started - arrived4assessment
            queued4assessmentOPD.append(queued)
            print("{} started assessment in outpatient care at {} after waiting {}".format(patient_ID, started, queued))            

            delta = random.expovariate(1.0 / mean_CT2assessOPD)
            yield env.timeout(delta)
    else:
        with doctorER.request() as req:
            # Wait until doctor is available for inpatient care
            yield req

            started = env.now
            queued = started - arrived4assessment
            queued4assessmentER.append(delta)
            print("{} started asessment in inpatient care at {} after waiting {}".format(patient_ID, started, queued))
            
            delta = random.expovariate(1.0 / mean_CT2assessER)
            yield env.timeout(delta)

            exited = env.now    
            TAT = exited - arrived
            leadTimes.append(TAT)
            print("{} HAD LEAD TIME OF {:.0f} MINUTES.".format(patient_ID, TAT))

                        

### 3. Simulate One Run

Take a test drive.

In [31]:
# Set up the simulation environment
env = simpy.Environment()

# Set up the resources
receptionist = simpy.Resource(env, capacity=1)
nurse = simpy.Resource(env, capacity=2)
doctorOPD = simpy.Resource(env, capacity=1)
doctorER = simpy.Resource(env, capacity=2)

# Configure simulation parameters
mean_IAT = 8
mean_CT2register = 2
mean_CT2triage = 5
mean_CT2assessOPD = 60
mean_CT2assessER = 30

# Lists of Globals
queued4registration = []
queued4triage = []
queued4assessmentOPD = []
queued4assessmentER = []
leadTimes = []

# Make it so
env.process(generate_arrivals(env, mean_IAT, mean_CT2register, mean_CT2triage, mean_CT2assessOPD, mean_CT2assessER,
                            receptionist, nurse, doctorOPD, doctorER))
env.run(until=480)

print(leadTimes)
print("Median time queued in | registration    | is {}".format(median(queued4registration)))
print("Median time queued in | triage          | is {}".format(median(queued4triage)))
print("Median time queued in | assessment OPD  | is {}".format(median(queued4assessmentOPD)))
print("Median time queued in | assessment ED   | is {}".format(median(queued4assessmentER)))
print("Median time queued in | START 2 FINISH  | is {}".format(median(leadTimes)))


0 arrived at 0.00
0 started registration at 0.00 after waiting 0.00
1 arrived at 3.18
2 arrived at 4.06
0 started triage at 4.10 after waiting 0.00
1 started registration at 4.10 after waiting 0.92
0 started asessment in inpatient care at 4.84974631948254 after waiting 0.0
1 started triage at 5.22 after waiting 0.00
2 started registration at 5.22 after waiting 1.15
2 started triage at 6.69 after waiting 0.00
1 started asessment in inpatient care at 9.566552195462235 after waiting 0.0
3 arrived at 16.30
3 started registration at 16.30 after waiting 0.00
3 started triage at 16.59 after waiting 0.00
1 HAD LEAD TIME OF 19 MINUTES.
3 started asessment in inpatient care at 22.34380231578365 after waiting 5.501542206190788
0 HAD LEAD TIME OF 25 MINUTES.
2 started asessment in inpatient care at 24.946294186669896 after waiting 3.859566526073401
2 HAD LEAD TIME OF 24 MINUTES.
3 HAD LEAD TIME OF 12 MINUTES.
4 arrived at 33.59
4 started registration at 33.59 after waiting 0.00
4 started triage at